In [8]:
using TwoFAST
using Dierckx
using DelimitedFiles
using QuadGK
using LinearAlgebra
using SpecialFunctions
using SparseArrays

# Loading the power spectrum to be projected onto the spherical harmonics:

In [9]:
path = homedir() * "/.julia/packages/TwoFAST/"
path *= readdir(path)[1]
path *= "/test/data/planck_base_plikHM_TTTEEE_lowTEB_lensing_post_BAO_H070p6_JLA_matterpower.dat"
d = readdlm(path, comments=true)
pk = Spline1D(d[:,1], d[:,2])


N = 1024    # number of points to use in the Fourier transform
kmax = 1e3  # maximum k-value
kmin = 1e-5 # minimum k-value
chi0 = 1e-3   # minimum r-value (should be ~1/kmax)
q = 1.1; #What is it exactly?!

#Neal's initial condition:
#N = 4096;
#kmin = 1e-5;
#kmax = 1e3;


# First one may want to calculate the two point correlation function as described below:
## $ \xi_l^v(r) = \int_0^{\infty} \frac{k^2 dk}{2\pi^2} P_k \frac{j_l(kr)}{(kr)^v} $


In [7]:

print("ξ(r), ℓ=0, ν=0: ")
r00, xi00 = xicalc(pk, 0, 0; N=N, kmin=kmin, kmax=kmax, r0=r0)

print("ξ(r), ℓ=0, ν=-2:")
r, xi0m2 = xicalc(pk, 0, -2; N=N, kmin=kmin, kmax=kmax, r0=r0)

ξ(r), ℓ=0, ν=0: ξ(r), ℓ=0, ν=-2:

([0.001, 0.0010181517217181819, 0.001036632928437698, 0.001055449600878603, 0.0010746078283213173, 0.0010941138105771857, 0.0011139738599948025, 0.0011341944035027569, 0.0011547819846894584, 0.001175743265920711  …  83536.25469578238, 85052.58154439961, 86596.43233600643, 88168.3066775569, 89768.71324473115, 91398.169946549, 93057.20409296977, 94746.35256553731, 96466.16199111959, 98217.18891880369], [6.101081902542977, 6.312688061711316, 6.53117938569655, 6.756745912201364, 6.989580029093844, 7.229876701968187, 7.477832837119251, 7.733647404183142, 7.997520790227869, 8.269654809492199  …  4.32671553356489, 4.479341491815969, 4.637127433553351, 4.80023116240264, 4.968813751891052, 5.1430400391844096, 5.323078054817007, 5.5090995047573585, 5.70127916904642, 5.89979535084241])

# Now one may want to calculate the $M_{ll}$ which is:

## The final step of projecting onto the spherical harmonics is to actually find the $C_l$ coefficients as follows:

## $C_l(r, r') = \frac{2}{\pi} \int k^2 dk \ P(k) \ j_l(kr) \ j_l(kr')$

## To calculate this one may want to first calculate the fourier transform of the multiplication of two spherical bessel functions:

## $ e^{q\sigma} j_l(\alpha e^{\sigma}) \ j_l'(\beta e^{\sigma}) = \int \frac{dt}{2 \pi} e^{it \sigma} M_{ll'}^{q}(t,R)$

In [12]:

nl=500; ell=[i for i in 1:nl ];  
nR=384; RR = [10^((i-nR)/512) for i in 1:nR ];

#TwoFAST example
#ell = [42]  # only ell=42 for this run
#RR = [0.6, 0.7, 0.8, 0.9, 1.0]

# calculate M_ll at high ell, result gets saved to a file:
#This calculates the multiplication of two besse functions
f21cache = F21EllCache(maximum(ell), RR, N; q=q, kmin=kmin, kmax=kmax, χ0=chi0)
write("out/F21EllCache", f21cache)

# calculate all M_ll, result gets saved to a file:
mlcache = MlCache(ell, "out/F21EllCache", "out/MlCache")
write("out/MlCache", mlcache)



  q=1.1, R=0.1786294763086179:	  2.709073 seconds (1.36 M allocations: 222.731 MiB, 16.83% gc time, 9.03% compilation time)
  q=1.1, R=0.17943462442492472:	  2.102622 seconds (1.32 M allocations: 218.877 MiB, 1.96% gc time)
  q=1.1, R=0.18024340163707053:	  1.994225 seconds (1.31 M allocations: 218.007 MiB, 1.97% gc time)
  q=1.1, R=0.1810558243027122:	  1.987641 seconds (1.30 M allocations: 216.260 MiB, 2.12% gc time)
  q=1.1, R=0.18187190885323648:	  1.898244 seconds (1.28 M allocations: 212.241 MiB, 1.86% gc time)
  q=1.1, R=0.18269167179409243:	  1.635117 seconds (1.27 M allocations: 210.641 MiB, 1.66% gc time)
  q=1.1, R=0.18351512970512512:	  1.545027 seconds (1.26 M allocations: 209.210 MiB, 2.31% gc time)
  q=1.1, R=0.18434229924091106:	  1.492048 seconds (1.25 M allocations: 206.884 MiB, 2.16% gc time)
  q=1.1, R=0.18517319713109495:	  2.024258 seconds (1.24 M allocations: 205.270 MiB, 1.76% gc time)
  q=1.1, R=0.1860078401807282:	  1.720928 seconds (1.22 M allocations: 202.34

  q=1.1, R=0.25713207233666546:	  0.240277 seconds (256.74 k allocations: 39.369 MiB, 2.72% gc time)
  q=1.1, R=0.2582910602481916:	  0.358785 seconds (246.50 k allocations: 37.644 MiB, 2.92% gc time)
  q=1.1, R=0.25945527214040154:	  0.402039 seconds (252.98 k allocations: 38.689 MiB, 3.04% gc time)
  q=1.1, R=0.260624731559679:	  0.249088 seconds (251.38 k allocations: 38.413 MiB, 2.65% gc time)
  q=1.1, R=0.26179946215854:	  0.289791 seconds (249.32 k allocations: 38.064 MiB, 2.97% gc time)
  q=1.1, R=0.2629794876961109:	  0.265093 seconds (238.95 k allocations: 36.345 MiB, 1.90% gc time)
  q=1.1, R=0.26416483203860924:	  0.224104 seconds (237.30 k allocations: 36.062 MiB)
  q=1.1, R=0.2653555191598262:	  0.230616 seconds (243.18 k allocations: 37.026 MiB, 2.72% gc time)
  q=1.1, R=0.2665515731416115:	  0.310292 seconds (231.46 k allocations: 35.087 MiB, 2.65% gc time)
  q=1.1, R=0.26775301817436076:	  0.323825 seconds (228.58 k allocations: 34.622 MiB, 3.37% gc time)
  q=1.1, R=0.2

  q=1.1, R=0.37516192015446387:	  0.180136 seconds (225.43 k allocations: 34.095 MiB, 3.85% gc time)
  q=1.1, R=0.37685291158300366:	  0.177919 seconds (225.44 k allocations: 34.095 MiB)
  q=1.1, R=0.37855152492586297:	  0.184438 seconds (235.91 k allocations: 35.833 MiB, 1.98% gc time)
  q=1.1, R=0.38025779453778613:	  0.285308 seconds (236.37 k allocations: 35.909 MiB, 2.01% gc time)
  q=1.1, R=0.3819717549283665:	  0.226648 seconds (236.30 k allocations: 35.888 MiB, 2.14% gc time)
  q=1.1, R=0.3836934407627449:	  0.247380 seconds (225.71 k allocations: 34.145 MiB, 2.50% gc time)
  q=1.1, R=0.38542288686231063:	  0.237991 seconds (225.50 k allocations: 34.102 MiB)
  q=1.1, R=0.3871601282054056:	  0.229618 seconds (247.09 k allocations: 37.665 MiB, 2.07% gc time)
  q=1.1, R=0.38890519992803174:	  0.315526 seconds (225.89 k allocations: 34.175 MiB, 2.16% gc time)
  q=1.1, R=0.39065813732456195:	  0.207087 seconds (225.49 k allocations: 34.115 MiB, 2.12% gc time)
  q=1.1, R=0.3924189758

  q=1.1, R=0.5473703262878217:	  0.397875 seconds (236.32 k allocations: 35.867 MiB, 2.01% gc time)
  q=1.1, R=0.5498375237304849:	  0.252273 seconds (235.48 k allocations: 35.726 MiB, 3.06% gc time)
  q=1.1, R=0.5523158417307099:	  0.318037 seconds (235.47 k allocations: 35.713 MiB, 1.44% gc time)
  q=1.1, R=0.5548053304129003:	  0.336703 seconds (235.86 k allocations: 35.785 MiB, 1.90% gc time)
  q=1.1, R=0.5573060401273886:	  0.364570 seconds (235.55 k allocations: 35.740 MiB)
  q=1.1, R=0.5598180214514549:	  0.277033 seconds (234.76 k allocations: 35.611 MiB, 1.91% gc time)
  q=1.1, R=0.5623413251903491:	  0.235001 seconds (234.88 k allocations: 35.624 MiB, 1.95% gc time)
  q=1.1, R=0.5648760023783191:	  0.300022 seconds (234.89 k allocations: 35.634 MiB, 1.43% gc time)
  q=1.1, R=0.5674221042796429:	  0.200439 seconds (234.87 k allocations: 35.630 MiB, 1.74% gc time)
  q=1.1, R=0.5699796823896648:	  0.191168 seconds (234.21 k allocations: 35.525 MiB)
  q=1.1, R=0.5725487884358379:

  q=1.1, R=0.8022262617851205:	  0.370405 seconds (227.46 k allocations: 34.432 MiB, 1.48% gc time)
  q=1.1, R=0.8058421877614819:	  0.277350 seconds (227.33 k allocations: 34.415 MiB, 1.53% gc time)
  q=1.1, R=0.8094744120333358:	  0.313554 seconds (227.52 k allocations: 34.449 MiB, 2.19% gc time)
  q=1.1, R=0.813123008063037:	  0.291089 seconds (227.19 k allocations: 34.395 MiB, 1.45% gc time)
  q=1.1, R=0.8167880496440613:	  0.340809 seconds (226.94 k allocations: 34.343 MiB)
  q=1.1, R=0.820469610902499:	  0.316663 seconds (227.04 k allocations: 34.360 MiB, 1.50% gc time)
  q=1.1, R=0.8241677662985538:	  0.297333 seconds (226.84 k allocations: 34.336 MiB, 1.44% gc time)
  q=1.1, R=0.827882590628048:	  0.364025 seconds (226.83 k allocations: 34.332 MiB, 2.50% gc time)
  q=1.1, R=0.8316141590239368:	  0.307872 seconds (226.72 k allocations: 34.310 MiB)
  q=1.1, R=0.8353625469578262:	  0.305176 seconds (226.82 k allocations: 34.333 MiB, 1.79% gc time)
  q=1.1, R=0.8391278302415006:	  

ell 424, elapsed: 0.5590000152587891
ell 423, elapsed: 0.6470000743865967
ell 422, elapsed: 0.6059999465942383
ell 421, elapsed: 0.5350000858306885
ell 420, elapsed: 0.48799991607666016
ell 419, elapsed: 2.996999979019165
ell 418, elapsed: 1.9140000343322754
ell 417, elapsed: 0.4160001277923584
ell 416, elapsed: 0.44899988174438477
ell 415, elapsed: 0.5900001525878906
ell 414, elapsed: 1.3249998092651367
ell 413, elapsed: 0.48600006103515625
ell 412, elapsed: 3.009000062942505
ell 411, elapsed: 2.936000108718872
ell 410, elapsed: 2.0929999351501465
ell 409, elapsed: 0.6110000610351562
ell 408, elapsed: 0.5659999847412109
ell 407, elapsed: 0.5910000801086426
ell 406, elapsed: 0.5230000019073486
ell 405, elapsed: 0.5789999961853027
ell 404, elapsed: 2.5460000038146973
ell 403, elapsed: 2.75600004196167
ell 402, elapsed: 0.6370000839233398
ell 401, elapsed: 0.5490000247955322
ell 400, elapsed: 0.5289998054504395
ell 399, elapsed: 0.5480000972747803
ell 398, elapsed: 0.5420000553131104
ell

ell 202, elapsed: 0.5160000324249268
ell 201, elapsed: 0.5469999313354492
ell 200, elapsed: 0.6389999389648438
ell 199, elapsed: 0.622999906539917
ell 198, elapsed: 3.31000018119812
ell 197, elapsed: 6.773000001907349
ell 196, elapsed: 0.44899988174438477
ell 195, elapsed: 0.9539999961853027
ell 194, elapsed: 0.440000057220459
ell 193, elapsed: 0.6390001773834229
ell 192, elapsed: 3.2159998416900635
ell 191, elapsed: 3.940999984741211
ell 190, elapsed: 0.6029999256134033
ell 189, elapsed: 0.7170000076293945
ell 188, elapsed: 0.5119998455047607
ell 187, elapsed: 0.4509999752044678
ell 186, elapsed: 0.46399998664855957
ell 185, elapsed: 2.5299999713897705
ell 184, elapsed: 2.6540000438690186
ell 183, elapsed: 1.694000005722046
ell 182, elapsed: 0.5379998683929443
ell 181, elapsed: 0.624000072479248
ell 180, elapsed: 0.8669998645782471
ell 179, elapsed: 0.7430000305175781
ell 178, elapsed: 0.8250000476837158
ell 177, elapsed: 7.468999862670898
ell 176, elapsed: 9.430999994277954
ell 175, 

200

## Now here we calculated the multiplication of two bessel functions. This can be used to read directly now and calculate the C_l's the coefficients of the spherical harmonics that we were looking for. 

### The net step is just here to help us smallen our arrays a little bit more by taking a look at the values of r!
## We just want the power spectrum on the scale $\ 1<r<10K\  MPc/h$ 

In [13]:
rr = calcwljj(pk, RR; ell=[-1], kmin=kmin, kmax=kmax, N=N, r0=chi0, q=q);

In [16]:
rr[1537]

0.9999999999999998

In [19]:
rr[3585]

9999.99999999997

In [20]:
#What does i exactly corresponds to here?
#we just want the rs between 1 to 10000Mpc/h
imin=1537;
imax=3585;
ra=rr[imin:imax];
nr=length(ra);
wl=zeros(nr,nr,nl);

LoadError: OutOfMemoryError()

# Actually calculating the $C_l$s

In [11]:
# calculate wljj:
w00 = Array{Float64}(undef, N, length(RR))
w02 = Array{Float64}(undef, N, length(RR))
function outfunc(wjj, ell, rr, RR)
    if ell == 42
        w00[:,:] = wjj[1]
        w02[:,:] = wjj[2]
    end
end
rr = calcwljj(pk, RR; ell=ell, kmin=kmin, kmax=kmax, N=N, r0=chi0, q=q, outfunc=outfunc, cachefile="out/MlCache/MlCache.bin")

100-element Vector{Float64}:
 0.6406792005704995
 0.6435669750977286
 0.6464677658766367
 0.6493816315762113
 0.6523086311298825
 0.6552488237367147
 0.6582022688626041
 0.6611690262414815
 0.6641491558765202
 0.6671427180413495
 0.670149773281274
 0.6731703824144982
 0.6762046065333562
 ⋮
 0.9517340888833214
 0.9560239010953075
 0.9603330490535164
 0.9646616199111993
 0.9690097012144389
 0.9733773809039202
 0.9777647473167089
 0.9821718891880378
 0.9865988956531019
 0.9910458562488609
 0.9955128609158502
 1.0